# Example for data science
Purpose of this exercise is download the dataset, save it to a database and answer few questions.
<br />
Dataset source <link>https://files.grouplens.org/datasets/movielens/ml-latest-small.zip</link>

### Data preparation 
1. Download dataset
2. Unzip dataset
3. Load the data
4. Save it to database

### Question to be answered : 
1. How many movies are in data set ?
2. What is the most common genre of movie?
3. What are top 10 movies with highest rate ?
4. What are 5 most often rating users ?
5. When was done first and last rate included in data set and what was the rated movie tittle?
6. Find all movies released in 1990

### Downloading dataset

In [1]:
import os
import requests

if os.path.exists('data') == False:
    os.mkdir('data')

url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
r = requests.get(url, allow_redirects=True, verify=False)
open('data/ml-latest-small.zip', 'wb').write(r.content)

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'files.grouplens.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


978202

### Unzip dataset

In [2]:
import zipfile

with zipfile.ZipFile('data/ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data/unzipped')

### Load the data
While loading data, we remove NAN values to have a pure dataset

In [3]:
from numpy import genfromtxt
import numpy as np
import csv

def Load_Data(file_name):
    with open(file_name, newline='') as f:
        print(f)
        reader = csv.reader(f)
        return list(reader)


In [4]:
def HasEmptyCell(row):
    for elem in row:
        if(elem == None or elem.strip() == ''):
            return True
    return False

### Save it to database

In [5]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, Date, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pymysql

In [6]:
Base = declarative_base()

#### Data models

In [7]:
class Movies(Base):
    __tablename__ = "movies"
    __table_args__ = {'sqlite_autoincrement': True}
    
    id = Column(Integer, primary_key=True, nullable=False)
    movieId = Column(Integer)
    title = Column(Text)
    genres = Column(Text)

In [8]:
class Links(Base):
    __tablename__ = "links"
    __table_args__ = {'sqlite_autoincrement': True}
    
    id = Column(Integer, primary_key=True, nullable=False)
    movieId = Column(Integer)
    imdbId = Column(Integer)
    tmdbId = Column(Float)

In [9]:
class Ratings(Base):
    __tablename__ = "ratings"
    __table_args__ = {'sqlite_autoincrement': True}
    
    id = Column(Integer, primary_key=True, nullable=False)
    userId = Column(Integer)
    movieId = Column(Integer)
    rating = Column(Float)
    timestamp = Column(Integer)

In [10]:
class Tags(Base):
    __tablename__ = "tags"
    __table_args__ = {'sqlite_autoincrement': True}
    
    id = Column(Integer, primary_key=True, nullable=False)
    userId = Column(Integer)
    movieId = Column(Integer)
    tag = Column(Text)
    timestamp = Column(Integer)

#### Lets create database if not exists

In [11]:
userName = "root"
password = "password"
ip = "mysql"
port = "3306"

engine = create_engine(f'mysql+pymysql://{userName}:{password}@{ip}:{port}')

In [12]:
dbName = "exercise"
engine.execute(f"CREATE DATABASE IF NOT EXISTS {dbName};")
engine = create_engine(f'mysql+pymysql://{userName}:{password}@{ip}:{port}/{dbName}') # engine recreated for simplycity

#### Initialize the models and create a session

In [13]:
Base.metadata.create_all(engine)

session = sessionmaker()
session.configure(bind=engine)
s = session()

#### Clean tables for the getting same data for each fresh run

In [14]:
s.query(Movies).delete()
s.query(Links).delete()
s.query(Ratings).delete()
s.query(Tags).delete()
s.commit()

#### NOTE: We do not include rows with empty values to have cleaner results

In [ ]:
for m in Load_Data('data/unzipped/ml-latest-small/movies.csv')[1:] :
    if (HasEmptyCell(m) == False):
        movie = Movies(**{
            'movieId' : m[0],
            'title' : m[1],
            'genres' : m[2],
        })
        s.add(movie)
s.commit()

<_io.TextIOWrapper name='data/unzipped/ml-latest-small/movies.csv' mode='r' encoding='UTF-8'>


In [ ]:
for m in Load_Data('data/unzipped/ml-latest-small/links.csv')[1:] :
    if (HasEmptyCell(m) == False):
        link = Links(**{
            'movieId' : m[0],
            'imdbId' : m[1],
            'tmdbId' : m[2],
        })
        s.add(link)
s.commit()

In [ ]:
for m in Load_Data('data/unzipped/ml-latest-small/ratings.csv')[1:] :
    if (HasEmptyCell(m) == False):
        rating = Ratings(**{
            'userId' : m[0],
            'movieId' : m[1],
            'rating' : m[2],
            'timestamp' : m[3],
        })
        s.add(rating)
s.commit()

In [ ]:
for m in Load_Data('data/unzipped/ml-latest-small/tags.csv')[1:] :
    if (HasEmptyCell(m) == False):
        tag = Tags(**{
            'userId' : m[0],
            'movieId' : m[1],
            'tag' : m[2],
            'timestamp' : m[3],
        })
        s.add(tag)
s.commit()

## Questions and Answers

### Question 1) How many movies are in data set ?

In [ ]:
sql_query = sqlalchemy.text("SELECT count(distinct( movies.title)) FROM exercise.movies as movies;")
result = engine.execute(sql_query)
numberOfDistinctMovieTitles = result.fetchone()[0]
print(f'Number of distinct movie titles is {numberOfDistinctMovieTitles}')

### Question 2) What is the most common genre of movie?

In [ ]:
sql_query = sqlalchemy.text("SELECT movies.genres FROM exercise.movies as movies;")
result = engine.execute(sql_query)
result_as_list = result.fetchall()
result_as_list[:10]

<br />
MySql does not support a straight forward function to split each string into different raws. So we take all the genres in the table and split them in memory using python's "split" function.
<br />

In [ ]:
counter = {}
for genreRaw in result_as_list:
    for g in genreRaw[0].split("|"):
        if g not in counter:
            counter[g] = 1
        else:
            counter[g] += 1
counter

In [ ]:
import operator

mostCommonGenre = max(counter.items(), key=operator.itemgetter(1))[0]

print(f'Most common genre is {mostCommonGenre}')

### Question 3) What are top 10 movies with highest rate ?

In [ ]:
sql_query = sqlalchemy.text("SELECT movies.title " + 
                            "FROM exercise.movies as movies " +
                            "join exercise.ratings as ratings On movies.movieId = ratings.movieId " + 
                            "Group by movies.title " + 
                            "ORDER BY AVG(ratings.rating) desc LIMIT 10;")
result = engine.execute(sql_query)
result_as_list = result.fetchall()
for r in result_as_list:
    print(r[0])

### Question 4) What are 5 most often rating users ?
'Most often' is a hard question to answer but "most" is simple.

In [ ]:
sql_query = sqlalchemy.text("SELECT userId, count(*) as frequent " + 
                            "FROM exercise.ratings " + 
                            "Group by userId " +
                            " ORDER BY frequent desc limit 5;")
result = engine.execute(sql_query)
result_as_list = result.fetchall()
for r in result_as_list:
    print(r)

### Question 5) When was done first and last rate included in data set and what was the rated movie tittle?

In [ ]:
sql_query = sqlalchemy.text("select min(ratings.timestamp)" +
                            "from exercise.ratings")
result = engine.execute(sql_query)
minTime = result.fetchone()
minTime[0]

In [ ]:
sql_query = sqlalchemy.text("select max(ratings.timestamp)" +
                            "from exercise.ratings")
result = engine.execute(sql_query)
maxTime = result.fetchone()
maxTime[0]

In [ ]:
def sql_GetMovieTitleWRTRatingTimestamp(timespamp):
    return sqlalchemy.text(f"select title " +
                            f"from exercise.movies as movies " +
                            f"where movies.movieId = " +
                            f"(" +
                            f"    select ratings.movieId " +
                            f"    from exercise.ratings as ratings " +
                            f"    where ratings.timestamp = " +
                            f" {timespamp} " +
                            f"    limit 1 " +
                            f");")

In [ ]:
sql_query = sql_GetMovieTitleWRTRatingTimestamp(minTime[0])
result = engine.execute(sql_query)
titleMin = result.fetchone()

In [ ]:
sql_query = sql_GetMovieTitleWRTRatingTimestamp(maxTime[0])
result = engine.execute(sql_query)
titleMax = result.fetchone()

In [ ]:
print (f"First movie rated {titleMin[0]} {minTime[0]}")
print (f"Last movie rated {titleMax[0]} {maxTime[0]}")

### Question 6) Find all movies released in 1990
Only place that we can receive movie release dates is in movies table's title column

In [ ]:
sql_query = sqlalchemy.text("select trim(left(movies.title,length(movies.title) - 6)) " +
                            "from exercise.movies as movies " +
                            "where substring(movies.title, -6) = '(1990)' ;")
result = engine.execute(sql_query)
moviesFrom1990 = result.fetchall()
moviesFrom1990